# Notebook for computing **TRUE FRECHET Similarities for Rome and Porto** 

In [ ]:
# Importing nescessary modules
import os, sys
import shutil

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

from utils.helpers import file_handler as fh
from utils.helpers import metafile_handler as mfh
from utils.similarity_measures import dtw, frechet

## CONSTANTS

In [8]:
NUMBER_OF_TRAJECTORIES = 50

#Rome
ROME_DATA_FOLDER = "../../../dataset/rome/output/"
ROME_DATA_META_FILE = f"{ROME_DATA_FOLDER}META-{NUMBER_OF_TRAJECTORIES}.txt"
ROME_SIMILARITY_VALUES_RESULT_FOLDER = "../../../results_true/similarity_values/rome/frechet"
ROME_FRECHET_FILENAME =  f"rome-frechet-{NUMBER_OF_TRAJECTORIES}.csv"


#Porto
PORTO_DATA_FOLDER  = "../../../dataset/porto/output/"
PORTO_DATA_META_FILE = f"{PORTO_DATA_FOLDER}/META-{NUMBER_OF_TRAJECTORIES}.txt"
PORTO_SIMILARITY_VALUES_RESULT_FOLDER = "../../../results_true/similarity_values/porto/frechet"
PORTO_FRECHET_FILENAME =  f"porto-frechet-{NUMBER_OF_TRAJECTORIES}.csv"

## FUNCTIONS

In [9]:
def deleteFile(file_name: str, folder_name: str) -> None:
    file_path = os.path.join(folder_name, file_name)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print("Failed to remove %s. Reason: %s" % (file_path, e))

In [10]:
# Using Cython Frechet, to speed things up


# NOTE - Is not in use due to extensive computation time for non parallel version.
# def generate_frechet_similarities(
#     data_folder: str, meta_file: str, file_name: str, similarities_output_folder: str
# ):
#     deleteFile(file_name, similarities_output_folder)

#     files = mfh.read_meta_file(meta_file)
#     trajectories = fh.load_trajectory_files(files, data_folder)

#     df = frechet.cy_frechet(trajectories)

#     df.to_csv(os.path.join(similarities_output_folder, file_name))


def generate_parallell_frechet_similarities(
    data_folder: str, meta_file: str, file_name: str, similarities_output_folder: str
):
    deleteFile(file_name, similarities_output_folder)

    files = mfh.read_meta_file(meta_file)
    trajectories = fh.load_trajectory_files(files, data_folder)

    df = frechet.cy_frechet_pool(trajectories)
    df.to_csv(os.path.join(similarities_output_folder, file_name))

## FRECHET SIMILARITIES FOR ROME


In [ ]:
# Test set
# generate_parallell_frechet_similarities(
#     DATA_ROME,
#     TEST_SET_ROME,
#     FRECHET_FILENAME_TEST_ROME,
#     SIMILARITIES_OUTPUT_FOLDER_ROME,
# )

# Full set
generate_parallell_frechet_similarities(
    ROME_DATA_FOLDER, ROME_DATA_META_FILE, ROME_FRECHET_FILENAME, ROME_SIMILARITY_VALUES_RESULT_FOLDER
)

## FRECHET SIMILARITIES FOR PORTO


In [ ]:
# Test set
# generate_parallell_frechet_similarities(
#     DATA_PORTO,
#     TEST_SET_PORTO,
#     FRECHET_FILENAME_TEST_PORTO,
#     SIMILARITIES_OUTPUT_FOLDER_PORTO,
# )

# Full set
generate_parallell_frechet_similarities(
    PORTO_DATA_FOLDER, PORTO_DATA_META_FILE, PORTO_FRECHET_FILENAME, PORTO_SIMILARITY_VALUES_RESULT_FOLDER
)